In [1]:
cd ..

d:\Project\ThesisProd


In [2]:
import os
import sys

import gradio as gr

from dotenv import load_dotenv
from core.retriever.Retriever import Retriever
from core.ingestion.preprocessing.storage.FaissStore import FaissStore
from core.llm.TeacherLLM import TeacherBot
from core.llm.AssistantLLM import AssistantBot

# Load environment variables from a .env file
load_dotenv()
# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

d:\Administrator\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Custom LLM initialized with model: gpt-3.5-turbo


In [3]:
os.path.exists("data/using/")

True

In [4]:
path = "data/using/"

In [5]:
faiss_store = FaissStore(documents_path = path)
nodes = faiss_store.get_nodes()

Loaded 640 documents.


In [6]:
store = Retriever(nodes=nodes)
retriever = store.get_retriever(top_k = 7)
context = store.get_big_context(retriever = retriever, query = "Machine Learning")

Successfully initialized for 2805 nodes.


In [8]:
llm = TeacherBot()
respond = llm.create_question(context = context)

Custom LLM initialized with model: gpt-3.5-turbo
[
["Question: What are some industries where machine learning is commonly used?","A. Retail","B. Education","C. Healthcare","D. Manufacturing","Answer: A. Retail, D. Manufacturing","Context: Machine learning in various industries"],
["Question: How do banks use machine learning in finance?","A. Credit applications","B. Stock market analysis","C. Crop cultivation","D. Weather prediction","Answer: A. Credit applications, B. Stock market analysis","Context: Machine learning in finance"],
["Question: What is one of the applications of machine learning in manufacturing?","A. Troubleshooting","B. Animal husbandry","C. Advertising","D. Fashion design","Answer: A. Troubleshooting","Context: Machine learning in manufacturing"],
["Question: In which field does machine learning aid in medical diagnosis?","A. Psychology","B. Neurology","C. Radiology","D. Optometry","Answer: C. Radiology","Context: Machine learning in medicine"],
["Question: How do t

In [9]:
print(context)

Machine learning is a versatile tool used in various industries such as retail, finance, manufacturing, medicine, telecommunications, and science. In finance, banks utilize machine learning for credit applications, fraud detection, and stock market analysis. In manufacturing, learning models are applied for optimization, control, and troubleshooting. In medicine, machine learning aids in medical diagnosis. Telecommunications companies analyze call patterns for network optimization and service quality improvement. In science, computers are essential for analyzing vast amounts of data in physics, astronomy, and biology. The World Wide Web's continuous growth necessitates automated information retrieval. Machine learning is not solely a database issue but also a component of artificial intelligence, enabling systems to learn and adapt to changing environments. It plays a crucial role in pattern recognition, vision, speech recognition, and robotics. Machine learning involves programming co

In [22]:
for i in range(len(respond)):
	for j in range(len(respond[i])):
		if j == 0:
			respond[i][0] = "###" + respond[i][0]
		if j == 5:
			respond[i][5] = ":green[" + respond[i][5] + "]"
		if j == 6:
			respond[i][6] = ":green[" + respond[i][6] + "]"

for i in range(len(respond)):
	for j in range(len(respond[i])):
		print(respond[i][j] + "\n")

#######Question: What are some industries where machine learning is commonly used?

A. Retail

B. Education

C. Healthcare

D. Manufacturing

:green[Answer: A. Retail, D. Manufacturing]

:green[Context: Machine learning in various industries]

#######Question: How do banks use machine learning in finance?

A. Credit applications

B. Stock market analysis

C. Crop cultivation

D. Weather prediction

:green[Answer: A. Credit applications, B. Stock market analysis]

:green[Context: Machine learning in finance]

#######Question: What is one of the applications of machine learning in manufacturing?

A. Troubleshooting

B. Animal husbandry

C. Advertising

D. Fashion design

:green[Answer: A. Troubleshooting]

:green[Context: Machine learning in manufacturing]

#######Question: In which field does machine learning aid in medical diagnosis?

A. Psychology

B. Neurology

C. Radiology

D. Optometry

:green[Answer: C. Radiology]

:green[Context: Machine learning in medicine]

#######Question: Ho